# SOHO analysis 
The purpose of this notebook is to analysis visual-spectrum images of the sun from the SOHO imaging satellite at the L1 Lagrange point (meaning the images are from the same perspective as the Earth) and use this data to determine the rotational period of the sun.

In [1]:
import os
from datetime import datetime

# Function to extract the image paths and their timestamps
def get_files_with_times(root_dir = "data"):
    file_paths = []
    times = []
    for day_dir in sorted(os.listdir(root_dir)):
        day_path = os.path.join(root_dir,day_dir)
        if os.path.isdir(day_path):
            for file in sorted(os.listdir(day_path)):
                if file.endswith(".jpg"):
                    time_str = file.split('_')[1] # Extract time (hhmm)
                    time = datetime.strptime(f"{day_dir}_{time_str}", r"%Y%m%d_%H%M")
                    file_paths.append(os.path.join(day_path,file))
                    times.append(time)
    return file_paths, times

file_paths, times = get_files_with_times()

In [2]:
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown
from utils.image_processing import detect_sunspots
import cv2

# Create a dropdown to select days
day_dirs = sorted([d for d in os.listdir("data") if os.path.isdir(os.path.join("data", d))])
@interact(day=Dropdown(options=day_dirs, description="Select Day:"))
def show_day_images(day):
    day_path = os.path.join("data", day)
    files = sorted([f for f in os.listdir(day_path) if f.endswith(".jpg")])[:16] #Only the first 12 images
    
    fig, axes = plt.subplots(3, 4, figsize=(15, 10))
    for ax, file in zip(axes.flat, files):
        img, centroids, solar_center, solar_radius = detect_sunspots(os.path.join(day_path, file))
        print(centroids)
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.scatter([c[0] for c in centroids], [c[1] for c in centroids], s=5, c='red')
        ax.set_title(file.split('_')[1])  # Show time (hhmm)
        ax.axis('off')
    plt.tight_layout()

interactive(children=(Dropdown(description='Select Day:', options=('20250422', '20250423', '20250424', '202504…

The cell above contains the detected features highlighted in red with a dropdown to select which day's data to look at.

In [3]:
from utils.feature_tracking import SunspotTracker

_, _, solar_center, solar_radius = detect_sunspots(file_paths[0]) #Initial value for solar radius from the first image
tracker = SunspotTracker(solar_center, solar_radius, 1)

#main feature tracking loop
for img, time in zip(file_paths, times):
    img, centroids, solar_center, solar_radius = detect_sunspots(img)
    
    tracker.process_frame(time, centroids)
velocities = tracker.get_all_velocities()
print(velocities)

[[-5.262828931422064, 0.9867187490399374, -1.3515859855619965, -1.409500339578699, -1.4632961411593897, -5.428834784772789, -1.5438763501279027, -1.5870378194053956, -1.626365676889236, -1.6619352273633012, -5.550552408690692, -1.7128651094149063, -1.7387318893042902, -1.7610461254234906, -5.608730935810854, 0.7549619391743363, -1.7874901924474216], [-1.6987378430803801, -1.615069209896319, 2.378547193111899, -1.4018982406314535, -1.302035615431123, -1.1968058137399566, 4.574473799716543, -0.9358521413187191, -0.8109270455875048, -0.6792334364272392, -0.540352287087444, -0.3938235238465495, 1.3269398073807679, -0.08459031355141633, 0.08764905253974575, 0.2698438710958726, -4.103013145829209, 0.598056760361942, 2.3297137340850895, -0.4828016594296969, -2.0651910848364423, 1.4324992876804572, 1.6982282576682337, 1.9818785387387834, -1.4357517997600553, 3.9459841824204887, -4.002947095298168, 5.951282557462946, -3.5894731136459086, 5.13151576807104, -1.6829379743949175, -0.062137806337887

In [5]:
tracker.tracks

[{'positions': [(231, 361),
   (233, 361),
   (236, 362),
   (239, 362),
   (242, 362),
   (245, 362),
   (247, 362),
   (250, 362),
   (253, 362),
   (256, 362),
   (259, 362),
   (261, 362),
   (264, 362),
   (267, 362),
   (270, 362),
   (272, 362),
   (275, 363),
   (278, 363)],
  'times': [datetime.datetime(2025, 4, 22, 9, 0),
   datetime.datetime(2025, 4, 22, 10, 30),
   datetime.datetime(2025, 4, 22, 12, 0),
   datetime.datetime(2025, 4, 22, 13, 30),
   datetime.datetime(2025, 4, 22, 15, 0),
   datetime.datetime(2025, 4, 22, 16, 30),
   datetime.datetime(2025, 4, 22, 18, 0),
   datetime.datetime(2025, 4, 22, 19, 30),
   datetime.datetime(2025, 4, 22, 21, 0),
   datetime.datetime(2025, 4, 22, 22, 30),
   datetime.datetime(2025, 4, 23, 0, 0),
   datetime.datetime(2025, 4, 23, 1, 30),
   datetime.datetime(2025, 4, 23, 3, 0),
   datetime.datetime(2025, 4, 23, 4, 30),
   datetime.datetime(2025, 4, 23, 6, 0),
   datetime.datetime(2025, 4, 23, 7, 30),
   datetime.datetime(2025, 4, 23, 